Имеется файл с временной статистикой работы асессоров над однотипным заданием. 

Формат файла: login tid Microtasks assigned_ts tclosed_ts 

Пояснение к формату: login — логин асессора; 
                     tid — id оцениваемого задания (task id); 
                     Microtasks – количество микрозаданий в одном задании; 
                     assigned_ts — время резервирования системой задания для асессора; 
                     closed_ts — точное время завершения работы над заданием; 
                     
                     разделитель — табуляция \t. 
                     
Задание может состоять из одного или несколько микрозаданий. Время резервирования задания (assigned_ts) указывает на тот момент, когда система назначила определенного асессора исполнителем этого задания. Этот момент может совпадать с временем начала работы асессора над заданием, а может и не совпадать (асессор может отойти выпить чаю, а потом приступить к заданию, асессор может выполнять предыдущее задание, в то время как за ним зарезервированы новые). 

Предположим, что асессор за 30 секунд своего рабочего времени получает N рублей. Какую оплату вы считаете справедливой для выполнения асессором одного микрозадания из этого файла? Опишите подробно все этапы вашего решения.

для выполнения асессором одного микрозадания

# Следовательно, нам не нужно давать оценку, ориентируясь на время

# Нужно взять данные, отбросить лишнее, рассчитать оптимальное время выполнения 1 микротаски и выразить данное время через N


In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data_task4_old.txt', sep='\t', index_col=0)

In [7]:
df.head(5)

,tid,Microtasks,assigned_ts,closed_ts
login,,,,
login0,190563850.0,4.0,2017-04-20 12:09:39,2017-04-20 13:13:01
login0,190561754.0,1.0,2017-04-20 12:10:30,2017-04-20 12:28:29
login0,190565906.0,4.0,2017-04-20 12:21:31,2017-04-20 13:30:10
login0,190560246.0,1.0,2017-04-20 12:28:30,2017-04-20 13:13:56
login0,190562168.0,2.0,2017-04-20 12:28:42,2017-04-20 13:14:50


Предварительный план:

1. Проверить гипотезу о том, что для каждому tid соответствует лишь одна запись о выполнении

Если гипотеза не верна, то сгруппировать по tid 
Если гипотеза верна то отобрать тестовый набор данных

Исходя из условий возможным вариантом решения считаю:
1. Рассчитать время, потраченное на каждую таску, построить график, оценить полученные данные

Предположительно, показатель времени, затраченного на задание, имеет нормальное распределение
Если это так, то нам следует рассчитать порог в 3 стандартных отконения, чтобы отбросить экстремальные значения

2. Рассчитать время, потраченное на каждую микротаску для каждого выполнения, построить график, оценить

In [4]:
print(len(df))

701827


In [8]:
test_data = df.head(100000)

In [13]:
test = df.groupby('tid')

In [14]:
%%time

print(len(test))

635044
Wall time: 8.6 s


In [15]:
task = df.groupby('Microtasks')

In [16]:
%%time

print(len(task))

50
Wall time: 73 ms


In [31]:
df.head(2)

,tid,Microtasks,assigned_ts,closed_ts,time
login,,,,,
login0,190563850.0,4.0,2017-04-20 12:09:39,2017-04-20 13:13:01,NaN
login0,190561754.0,1.0,2017-04-20 12:10:30,2017-04-20 12:28:29,NaN


In [32]:
df['time'] = np.nan

In [41]:
%%time

x = pd.to_datetime(df['assigned_ts'])
pd.to_datetime(df['closed_ts'])

Wall time: 2.1 s


In [40]:
df['time'] = pd.to_datetime(df['closed_ts']) - pd.to_datetime(df['assigned_ts'])

In [46]:
df.tail(5)

,tid,Microtasks,assigned_ts,closed_ts,time
login,,,,,
login766,195656026.0,1.0,2017-05-15 15:53:27,2017-05-15 16:16:56,00:23:29
login766,195656174.0,1.0,2017-05-15 15:53:33,2017-05-15 16:21:53,00:28:20
login766,195656466.0,1.0,2017-05-15 15:53:38,2017-05-15 16:23:41,00:30:03
login766,195656336.0,3.0,2017-05-15 15:54:18,2017-05-15 16:32:11,00:37:53
login766,195656078.0,2.0,2017-05-15 15:54:59,2017-05-15 16:17:32,00:22:33


In [53]:
df.sort_values(by = ['time'], ascending=False).head(5)

,tid,Microtasks,assigned_ts,closed_ts,time
login,,,,,
login657,185714006.0,7.0,2017-04-27 06:33:43,2017-05-20 22:06:41,23 days 15:32:58
login657,185714006.0,7.0,2017-04-27 06:33:43,2017-05-20 22:06:41,23 days 15:32:58
login300,185713854.0,17.0,2017-04-29 06:33:09,2017-05-22 16:21:41,23 days 09:48:32
login300,185713854.0,17.0,2017-04-29 06:33:09,2017-05-22 16:21:41,23 days 09:48:32
login300,189300182.0,13.0,2017-04-30 06:32:42,2017-05-22 16:23:44,22 days 09:51:02


In [ ]:
Как видно из отсортированных данных, изучаемый датасет имеет повторения, от которых следует избавиться

In [57]:
dfclean = df.drop_duplicates()
print(len(dfclean))

680037


In [58]:
dfclean.sort_values(by = ['time'], ascending=False).head(5)

,tid,Microtasks,assigned_ts,closed_ts,time
login,,,,,
login657,185714006.0,7.0,2017-04-27 06:33:43,2017-05-20 22:06:41,23 days 15:32:58
login300,185713854.0,17.0,2017-04-29 06:33:09,2017-05-22 16:21:41,23 days 09:48:32
login300,189300182.0,13.0,2017-04-30 06:32:42,2017-05-22 16:23:44,22 days 09:51:02
login300,185713868.0,15.0,2017-05-03 06:32:56,2017-05-22 16:26:19,19 days 09:53:23
login657,195642338.0,1.0,2017-05-17 06:41:20,2017-05-29 17:03:56,12 days 10:22:36
